## solve linear equations

### how do you describe and compute the vectors in the spaces?

turning the idea into an algorithm - what is the algorithm for
solving $Ax = 0$

example:

$A = \begin{bmatrix}
1 & 2 & 2 & 2 \\
2 & 4 & 6 & 8 \\
3 & 6 & 8 & 10
\end{bmatrix}$




column-2 and row-3 are not independent, because

column-2 is a multiple of column-1

row-3 is the sum of row-1 and row-2 

the vectors of these are in the same direction

we extend the algorithm of elimination to cover the case where
the pivot is zero; we just move on

while doing the elimination **the nullspace is unchanged**. but we
are chaning the column space.


after the first step of elimination, $A$ becomes

$\begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 2 & 4
\end{bmatrix}$

the pivot position $A_{22}$ is zero, and the position below it
$A_{32}$ is also zero,

it a telling sign that column-2 depends on an earlier column (hence
the two vertical zeros)

therefore we move on to the next good pivot, $A_{23}$

$\begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 0 & 0
\end{bmatrix}$

this is $U$, or upper echelon form

source: <https://en.wikipedia.org/wiki/Row_echelon_form>



### the rank of a matrix

the number of the pivots, $2$ is the rank of this matrix, $A$

the columns these pivots reside are **pivot columns**

we transformed $Ax = 0$ to $Ux = 0$ while preserving the solutions
and nullspace

the other columns are called **free columns** - because the
variables associated with these columns can hold any values

for example I can assign $x_{2}, x_{4}$ with 
$\begin{bmatrix}1 \\ 0 \end{bmatrix}$ - the choice zero is purely
for convenience (as any value works)



the algorithm to get the solutions is to separate the **pivot
columns**

plug $x_{2}$ and $x_{4}$ in the remaining equations:

$x_{1} + x_2{} * 2 + x_{3} * 2 + x_{4} * 2 = 0 \\
x_{3} * 2 + x_{4} * 4 = 0
$

we solve $x_{1}, x_{3}$ `(-2, 0)` and get the solution

$C = \begin{bmatrix}
-2 \\ 1 \\ 0 \\ 0
\end{bmatrix}$

#### where are the other solutions ?

1) we can take multiples of this particular solution, $C$, forming
a line in this 4-D space

2) assign other values (such as `(0, 1)`) to the free variables $x_{2}, x_{4}$


$\begin{align}
X = c\begin{bmatrix}-2 \\ 1 \\ 0 \\ 0\end{bmatrix} +
d\begin{bmatrix}2 \\ 0 \\ -2 \\ 1\end{bmatrix}
\end{align}$

the "special" values `(1, 0), (0, 1)` are chosen, then the rest
of the solutions of the nullspace are the linear combinations of
these two solutions, which solve $Ux = 0$


#### rank and the number of solutions

in the above case $A$, $r = 2$, meaning that there are really just
2 equations. Therefore the number of pivot columns is 2 and 
the number of free vars is $4 - 2 = 2$

generalising this to:

for an $n \times m$ matrix, the free variables are $n - r$


### reduced row echelon form, R

recall that the row echelon form of $A$ is

$A = \begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 0 & 0
\end{bmatrix}$

row-3 contains only zeros because original it was a linear combination
of some eariler rows (row-1 and row-2)

**reduced row echelon form has 0 above and below pivots**

$R = \begin{bmatrix}
1 & 2 & 0 & -2 \\
0 & 0 & 1 & 2 \\
0 & 0 & 0 & 0
\end{bmatrix}$



In [8]:
import sympy
import numpy as np

# source:
# https://stackoverflow.com/questions/7664246/python-built-in-function-to-do-matrix-reduction

A_ = np.array([
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
])

A = sympy.Matrix(A_)

# Matlab also offers rref() function

A.rref()

(Matrix([
 [1, 2, 0, -2],
 [0, 0, 1,  2],
 [0, 0, 0,  0]]),
 (0, 2))

#### information encoded in the RREF

1) pivot rows, pivot columns, which form an $I$ (which left a "free"
part of the matrix, $F$)

2) cleaned-up free columns

3) $Rx = 0$

(recap: from $Ax = 0$ to $Ux = 0$, to $Rx = 0$)

#### how to interpret RREF

a typical RREF 

$R = \begin{bmatrix}
I & F \\
0* & 0*
\end{bmatrix}$

(the zero-rows are optional)

that contains $r$ pivot rows, and $r$ pivot columns, and
$n - r$ free columns

construct a nullspace solution out of RREF

**using purely the matrix multiplication rules 
(see the block-multiplication
breakdown example in the previous chapter:
18.06_03_multiplication_and_inverse)**

I can derive a formula


$\begin{align}
RN = 0
\\
N = \begin{bmatrix}
-F \\ I
\end{bmatrix}
\end{align}$

In [59]:
# implement this formula,
# using sympy's Matrix class
# Matlab appears to have a command for computing N

import sympy
import numpy as np
from pprint import pprint as pp
A_ = np.array([
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
])
A = sympy.Matrix(A_)

# Return reduced row-echelon form of matrix and 
# indices of pivot vars.
R, iColumns = A.rref()
pp(R)


I = np.identity(len(iColumns))
F_ = []
for rowIdx in range(R.rows):
    row = R.row(rowIdx)
    if sum(row) == 0:
        continue
    cols = [x for colIdx, x in enumerate(row) if colIdx not in iColumns]
    F_.append(cols)

F = np.array(F_)
Finv = F * -1

# N
# each column of N holds a solution {x1, x2, x3, x4}
print(Finv)
print(np.array(I))

# it's good to iterate over the shape of F-inv
# I've added another example below when F-inv has less columns
# than I;
# the lecturer also shows that example in the video
for idx in range(Finv.shape[1]):
    x1 = Finv[0][idx]
    x2 = Finv[1][idx]
    x3 = I[0][idx]
    x4 = I[1][idx]
    print(f'{x1}, {x2}, {x3}, {x4}')
    

Matrix([
[1, 2, 0, -2],
[0, 0, 1,  2],
[0, 0, 0,  0]])
[[-2 2]
 [0 -2]]
[[1. 0.]
 [0. 1.]]
-2, 0, 1.0, 0.0
2, -2, 0.0, 1.0


#### back substitution on the reduced form

$\begin{align}
Rx = 0
\\
\begin{bmatrix}
I & F
\end{bmatrix} \ 
\begin{bmatrix}
x_{pivot} \\ x_{free}
\end{bmatrix}
= 0 \\
Rx = 0 = I x_{pivot} + F x_{free}
\\
x_{pivot} + Fx_{free} = 0
\end{align}$


if I choose to assign $I$ to $x_{free}$, then the whole equation
can be simplified to:

$x_{pivot} + F = 0\\
x_{pivot} = -F$

In [61]:
# transpose A and get a 3 x 4 matrix
# the algorithm still applies

import sympy
import numpy as np
from pprint import pprint as pp
A_ = np.array([
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
])
A = sympy.Matrix(A_.transpose())
R, iColumns = A.rref()
pp(R)
I = np.identity(len(iColumns))
F_ = []
for rowIdx in range(R.rows):
    row = R.row(rowIdx)
    if sum(row) == 0:
        continue
    cols = [x for colIdx, x in enumerate(row) if colIdx not in iColumns]
    F_.append(cols)
F = np.array(F_)
Finv = F * -1

print(Finv)
print(np.array(I))

for idx in range(Finv.shape[1]):
    x1 = Finv[0][idx]
    x2 = Finv[1][idx]
    x3 = I[0][idx]
    x4 = I[1][idx]
    print(f'{x1}, {x2}, {x3}, {x4}')

Matrix([
[1, 0, 1],
[0, 1, 1],
[0, 0, 0],
[0, 0, 0]])
[[-1]
 [-1]]
[[1. 0.]
 [0. 1.]]
-1, -1, 1.0, 0.0
